# ComposeML
### Getting Started
In this example, we have a data frame of transactions from different customers. To get an idea on how the data looks, we preview the data frame.

In [1]:
from featuretools.demo import load_mock_customer
df = load_mock_customer(return_single_table=True)
df.set_index('transaction_time', inplace=True)

df[df.columns[:8]].head()

,transaction_id,session_id,product_id,amount,customer_id,device,session_start,zip_code
transaction_time,,,,,,,,
2014-01-01 00:00:00,298,1,5,127.64,2,desktop,2014-01-01 00:00:00,13244
2014-01-01 00:09:45,10,1,5,57.39,2,desktop,2014-01-01 00:00:00,13244
2014-01-01 00:14:05,495,1,5,69.45,2,desktop,2014-01-01 00:00:00,13244
2014-01-01 02:33:50,460,10,5,123.19,2,tablet,2014-01-01 02:31:40,13244
2014-01-01 02:37:05,302,10,5,64.47,2,tablet,2014-01-01 02:31:40,13244


We want to extract label times for each customer where the label equals the total purchase amount over the next hour of transactions. First, we define a function that will return the total purchase amount given an hour of transactions.

In [2]:
def my_labeling_function(df_slice):
    label = df_slice["amount"].sum()
    return label

With the labeling function, we create `LabelMaker` to search and extract the labels from the data frame.

In [3]:
from composeml import LabelMaker

label_maker = LabelMaker(
    target_entity="customer_id",
    time_index="transaction_time",
    labeling_function=my_labeling_function,
    window_size="2h",
)

In [4]:
label_times = label_maker.search(
    df,
    minimum_data="1h",
    num_examples_per_instance=2,
    gap="2h",
)

label_times.head()

my_labeling_function
customer_id time                                     
1           2014-01-01 01:45:30               2077.66
            2014-01-01 03:45:30                943.28
2           2014-01-01 01:00:00               1320.64
            2014-01-01 03:00:00               2605.61
3           2014-01-01 02:45:05                944.85

Next, we take those label times and transform them into binary labels if the total amout was above 1000.

In [5]:
label_times = label_times.threshold(1000)

label_times.head()

my_labeling_function
customer_id time                                     
1           2014-01-01 01:45:30                  True
            2014-01-01 03:45:30                 False
2           2014-01-01 01:00:00                  True
            2014-01-01 03:00:00                  True
3           2014-01-01 02:45:05                 False

We could also take those label times and shift the time 1 hour earlier, so we can predict in advance.

In [6]:
label_times = label_times.apply_lead('1h')

label_times.head()

my_labeling_function
customer_id time                                     
1           2014-01-01 00:45:30                  True
            2014-01-01 02:45:30                 False
2           2014-01-01 00:00:00                  True
            2014-01-01 02:00:00                  True
3           2014-01-01 01:45:05                 False